<a href="https://colab.research.google.com/github/banno-0720/Deep-Learning-Projects/blob/main/AI_Image_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI IMAGE GENERATOR(ZERO SHOT INFERENCE)
  This notebook shows how to:
 1. Install dependencies  
 2. Load and run the Stable Diffusion v1.5 zero‑shot pipeline  
 3. Generate local sample images  
 4. Programmatically create a Hugging Face Space and push a Gradio demo  
 5. Serve the demo directly in this notebook  

 **Prerequisites**:  
 - You have a valid HF token with write access (set in Colab via `userdata.set('HF_TOKEN', '<your-token>')`)
 - Your HF username is `HimanshuGoyal2004`

## 1. 🔧 Install Required Libraries
 We need Diffusers, Transformers, Torch, Accelerate, XFormers (optional), Gradio, and the Hugging Face Hub client.


In [ ]:
!pip install -U diffusers transformers torch torchvision accelerate xformers gradio huggingface_hub

## 2. 🔑 Import Neccessary Libraries & Authenticate to Hugging Face
 Import necessary modules and log in using your HF token stored in Colab's user data.

In [ ]:
import os
import torch
from google.colab import userdata
from diffusers import AutoPipelineForText2Image
from huggingface_hub import login, create_repo, get_full_repo_name, upload_folder

In [ ]:
# Retrieve HF token from Colab userdata
HF_TOKEN = userdata.get('HF_TOKEN')
if not HF_TOKEN or not HF_TOKEN.startswith("hf_"):
    raise ValueError("Please store your HF token in Colab via userdata.set('HF_TOKEN', '<token>')")

In [ ]:
# Log in to Hugging Face(Optional)
# login(token=HF_TOKEN)

 ## 3. 🚀 Load Zero-Shot Stable Diffusion Pipeline
Load the pretrained Stable Diffusion v1.5 pipeline in FP16 on GPU (or FP32 on CPU).  
This lets us generate images from text prompts without any fine-tuning.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

# Load the pipeline
pipeline = AutoPipelineForText2Image.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype
).to(device)

print(f"🚩 Loaded pipeline on {device} with dtype={dtype}")

## 4. 🎨 Generate & Save Local Samples
 Run zero-shot generation on a few example prompts and save images locally.

In [ ]:
prompts = [
    "A serene mountain lake at sunrise",
    "A futuristic city skyline at night",
    "An astronaut sculpting a statue on the moon",
]

for i, prompt in enumerate(prompts, 1):
    img = pipeline(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
    fname = f"zero_shot_{i}.png"
    img.save(fname)
    print(f"✅ Saved `{fname}` for prompt: “{prompt}”")

## 5. 🏗️  Prepare Demo Folder for Your Space
 We will build a local folder `./sd_zero_shot_space/` containing:
 - `app.py` (Gradio demo)
 - `requirements.txt`
 - `README.md` (with YAML front‑matter for Space config)


In [ ]:
LOCAL_DEMO_FOLDER_PATH = "./sd_zero_shot_space"
HF_USERNAME            = "HimanshuGoyal2004"
HF_SPACE_NAME          = "sd-zero-shot-demo"
HF_REPO_ID             = f"{HF_USERNAME}/{HF_SPACE_NAME}"
HF_REPO_TYPE           = "space"
HF_SPACE_SDK           = "gradio"
HF_PRIVATE             = False

os.makedirs(LOCAL_DEMO_FOLDER_PATH, exist_ok=True)

# ── app.py ───────────────────────────────────────────────────────────────────
app_py = """
import gradio as gr
import torch
from diffusers import AutoPipelineForText2Image

# Detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype  = torch.float16 if device=='cuda' else torch.float32

# Load pipeline
pipe = AutoPipelineForText2Image.from_pretrained(
    'runwayml/stable-diffusion-v1-5', torch_dtype=dtype
).to(device)

def generate(prompt):
    try:
        out = pipe(prompt, guidance_scale=7.5, num_inference_steps=30)
        return out.images[0]
    except Exception as e:
        return f\"Error: {e}\"

# Build Gradio interface
gr.Interface(
    fn=generate,
    inputs=gr.Textbox(lines=2, placeholder='Enter prompt here...'),
    outputs='image',
    title='Stable Diffusion v1.5 Zero‑Shot',
    description='Generate images from text (zero‑shot).',
    allow_flagging='never'
).launch()
"""
with open(f"{LOCAL_DEMO_FOLDER_PATH}/app.py", "w") as f:
    f.write(app_py.lstrip())

# ── requirements.txt ─────────────────────────────────────────────────────────
requirements_txt = """
torch
diffusers
transformers
accelerate
gradio
huggingface_hub
"""
with open(f"{LOCAL_DEMO_FOLDER_PATH}/requirements.txt", "w") as f:
    f.write(requirements_txt.lstrip())

# ── README.md ─────────────────────────────────────────────────────────────────
readme_md = f"""
---
title: "Zero‑Shot Stable Diffusion Demo"
emoji: "🎨"
colorFrom: "blue"
colorTo: "purple"
sdk: "gradio"
sdk_version: "5.34.0"
app_file: "app.py"
pinned: false
---

# {HF_SPACE_NAME}

A zero‑shot Stable Diffusion v1.5 demo via Gradio.

- **Model**: `runwayml/stable-diffusion-v1-5`
- **Inference**: Zero‑shot text→image

Try it out live on Spaces!
"""
with open(f"{LOCAL_DEMO_FOLDER_PATH}/README.md", "w") as f:
    f.write(readme_md.lstrip())

print(f"🗂️ Prepared demo folder at {LOCAL_DEMO_FOLDER_PATH}")

## 6. 🌐 Create & Upload Your Hugging Face Space
Programmatically create a new public Gradio Space and push the demo folder into it.

In [ ]:
# Create the Space repo (idempotent)

create_repo(
    repo_id   = HF_REPO_ID,
    repo_type = HF_REPO_TYPE,
    private   = HF_PRIVATE,
    space_sdk = HF_SPACE_SDK,
    exist_ok  = True
)
print(f"🔗 Created Space repo: {HF_REPO_ID}")

# Upload all files from our demo folder
upload_folder(
    repo_id     = HF_REPO_ID,
    folder_path = LOCAL_DEMO_FOLDER_PATH,
    path_in_repo= "",
    repo_type   = HF_REPO_TYPE,
    ignore_patterns=["*.pyc", "__pycache__"]
)
print(f"📡 Uploaded demo to https://huggingface.co/spaces/{HF_REPO_ID}")

## 7. 🔧 (Optional) Enable GPU in Your Space
 1. Go to your Space: https://huggingface.co/spaces/{HF_REPO_ID}  
 2. Click **Settings → Hardware → Runtime type**  
 3. Select **GPU (T4)** for faster inference  
 4. Rebuild and try the demo

## 8. 🎨 Zero-Shot Inference with Gradio Blocks

In this cell, we build a **Gradio Blocks** interface for zero-shot text-to-image generation using our Stable Diffusion pipeline:

1. **Device & dtype detection**  
   We check for a GPU; if available we use **FP16**, otherwise we fall back to **FP32** on CPU.

2. **Pipeline loading**  
   We load the pretrained `runwayml/stable-diffusion-v1-5` model into our chosen dtype and device.

3. **`generate(prompt)` function**  
   - Takes a user text prompt  
   - Runs the diffusion pipeline with guidance  
   - Returns the generated `PIL.Image` or an error message

4. **Gradio Blocks layout**  
   - A Markdown header  
   - A row containing:  
     - A **Textbox** for prompt input  
     - A **Button** to trigger generation  
   - An **Image** output component

5. **Launch**  
   The interface appears inline in Colab, allowing you to enter text and see results directly.

In [ ]:
import gradio as gr
import torch
from diffusers import AutoPipelineForText2Image

# 1️⃣ Detect device and set dtype
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device == "cuda" else torch.float32

# 2️⃣ Load the pretrained pipeline in the correct dtype
pipe = AutoPipelineForText2Image.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=dtype
).to(device)

# 3️⃣ Define the generation function
def generate(prompt):
    try:
        # Run inference with classifier-free guidance
        out = pipe(
            prompt,
            guidance_scale=7.5,       # Strength of guidance
            num_inference_steps=30    # Number of diffusion steps
        )
        return out.images[0]         # Return the generated image
    except Exception as e:
        # If something goes wrong, return the error text
        return f"Error during generation:\n{e}"

# 4️⃣ Build the Gradio Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("## 🎨 Zero-Shot Stable Diffusion Demo")
    with gr.Row():
        inp = gr.Textbox(label="Prompt")   # Input textbox for user prompt
        btn = gr.Button("Generate")        # Button to trigger generation
    out = gr.Image(label="Output")         # Image component for display
    btn.click(fn=generate, inputs=inp, outputs=out)

# 5️⃣ Launch the interface inline
demo.launch()